In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
song_origin=pd.read_csv("sample_songlist.csv",index_col=0)
song_origin.head()

,Title,Artist,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,Emotion,Proba
0,사랑하기 싫어,지아,발라드,숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,슬픔,0.91
1,나비무덤,포맨 (4MEN),발라드,Standing beside you as sleep Wipe my tears as ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",Standing beside you as sleep Wipe my tears as ...,놀람공포,0.84
2,사랑인가 봐,멜로망스,"발라드, 국내드라마",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,놀람공포,0.85
3,너의 모든 순간,성시경,"발라드, 국내드라마",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,슬픔,0.66
4,사랑은 늘 도망가,임영웅,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,놀람공포,0.48


In [3]:
song_origin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3588 entries, 0 to 3587
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  3588 non-null   object 
 1   Artist                 3588 non-null   object 
 2   Genre                  3588 non-null   object 
 3   Lyric                  3588 non-null   object 
 4   Lyric_ver2             3588 non-null   object 
 5   preprocess_Lyric_ver2  3588 non-null   object 
 6   Emotion                3588 non-null   object 
 7   Proba                  3588 non-null   float64
dtypes: float64(1), object(7)
memory usage: 252.3+ KB


In [4]:
song_tag=pd.read_csv("df_final_lyrics.csv",index_col=0)
song_tag.head()

,Title,SongID,Artist,Date,Genre,Lyrics,Emotion,Probability,분노혐오,놀람공포,슬픔,행복
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"['숨을 내쉴 때마다', '눈을 감을 때마다', '니 생각이 나 먹먹해진다', '하...",슬픔,0.91,0.01,0.07,0.91,0.01
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"['Standing beside you as sleep', 'Wipe my tear...",놀람공포,0.84,0.02,0.84,0.12,0.02
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","['너와 함께 하고 싶은 일들을', '상상하는 게', '요즘 내 일상이 되고', '...",놀람공포,0.85,0.02,0.85,0.12,0.01
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","['이윽고 내가 한눈에', '너를 알아봤을 때', '모든 건 분명 달라지고 있었어'...",슬픔,0.66,0.02,0.30,0.66,0.02
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지...",놀람공포,0.48,0.01,0.48,0.45,0.06


In [5]:
#노래 & 가수가 같은 중복 데이터 제거
song_tag.drop_duplicates(subset=["Title","Artist"],inplace=True)
song_origin.drop_duplicates(subset=["Title","Artist"],inplace=True)

In [6]:
#인덱스 초기화
song_tag=song_tag.reset_index(drop=True)
song_origin=song_origin.reset_index(drop=True)

In [7]:
#노래 & 가수 열을 기준으로 데이터 합치기
song_total=pd.merge(song_tag,song_origin,on=["Title","Artist"],how="inner")

In [8]:
song_total.reset_index(drop=True,inplace=True)

In [9]:
song_total

,Title,SongID,Artist,Date,Genre_x,Lyrics,Emotion_x,Probability,분노혐오,놀람공포,슬픔,행복,Genre_y,Lyric,Lyric_ver2,preprocess_Lyric_ver2,Emotion_y,Proba
0,사랑하기 싫어,36206208,지아,2023.02.24,발라드,"['숨을 내쉴 때마다', '눈을 감을 때마다', '니 생각이 나 먹먹해진다', '하...",슬픔,0.91,0.01,0.07,0.91,0.01,발라드,숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...,슬픔,0.91
1,나비무덤,36235518,포맨 (4MEN),2023.03.01,발라드,"['Standing beside you as sleep', 'Wipe my tear...",놀람공포,0.84,0.02,0.84,0.12,0.02,발라드,Standing beside you as sleep Wipe my tears as ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",Standing beside you as sleep Wipe my tears as ...,놀람공포,0.84
2,사랑인가 봐,34657844,멜로망스,2022.02.18,"발라드, 국내드라마","['너와 함께 하고 싶은 일들을', '상상하는 게', '요즘 내 일상이 되고', '...",놀람공포,0.85,0.02,0.85,0.12,0.01,"발라드, 국내드라마",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,놀람공포,0.85
3,너의 모든 순간,4446485,성시경,2014.02.12,"발라드, 국내드라마","['이윽고 내가 한눈에', '너를 알아봤을 때', '모든 건 분명 달라지고 있었어'...",슬픔,0.66,0.02,0.30,0.66,0.02,"발라드, 국내드라마",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...,슬픔,0.66
4,사랑은 늘 도망가,34061322,임영웅,2021.10.11,"발라드, 국내드라마","['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지...",놀람공포,0.48,0.01,0.48,0.45,0.06,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,놀람공포,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,불티,36172278,최수호,2023.02.17,성인가요/트로트,"['나의 뜨거운 마음을', '불같은 나의 마음을', '다시 태울 수 없을까', '헤...",슬픔,0.95,0.02,0.02,0.95,0.01,성인가요/트로트,나의 뜨거운 마음을 불같은 나의 마음을 다시 태울 수 없을까 헤어지기는 정말 싫어 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",나의 뜨거운 마음을 불같은 나의 마음을 다시 태울 수 없을까 헤어지기는 정말 싫어 ...,슬픔,0.95
3542,안동역에서,32599795,임영웅,2020.05.15,성인가요/트로트,"['바람에 날려버린 허무한 맹세였나', '첫눈이 내리는 날 안동역 앞에서', '만나...",놀람공포,0.92,0.02,0.92,0.03,0.03,성인가요/트로트,바람에 날려버린 허무한 맹세였나 첫눈이 내리는 날 안동역 앞에서 만나자고 약속한사람...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",바람에 날려버린 허무한 맹세였나 첫눈이 내리는 날 안동역 앞에서 만나자고 약속한사람...,놀람공포,0.92
3543,남행열차,83059,김수희,1989.11,성인가요/트로트,"['비 내리는 호남선 남행열차에 ', '흔들리는 차창 너머로 ', '빗물이 흐르고 ...",놀람공포,0.72,0.01,0.72,0.20,0.07,성인가요/트로트,비 내리는 호남선 남행열차에 흔들리는 차창 너머로 빗물이 흐르고 내 눈물도 흐르...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",비 내리는 호남선 남행열차에 흔들리는 차창 너머로 빗물이 흐르고 내 눈물도 흐르...,놀람공포,0.72
3544,킬리만자로의 표범,81123,조용필,1985.11.15,성인가요/트로트,"['먹이를 찾아', '산 기슭을 어슬렁 거리는', '하이에나를 본일이 있는가 ', ...",놀람공포,0.71,0.19,0.71,0.10,0.01,성인가요/트로트,먹이를 찾아 산 기슭을 어슬렁 거리는 하이에나를 본일이 있는가 짐승의 썩은 고기만...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",먹이를 찾아 산 기슭을 어슬렁 거리는 하이에나를 본일이 있는가 짐승의 썩은 고기만...,놀람공포,0.71


In [10]:
song_total.columns

Index(['Title', 'SongID', 'Artist', 'Date', 'Genre_x', 'Lyrics', 'Emotion_x',
       'Probability', '분노혐오', '놀람공포', '슬픔', '행복', 'Genre_y', 'Lyric',
       'Lyric_ver2', 'preprocess_Lyric_ver2', 'Emotion_y', 'Proba'],
      dtype='object')

In [11]:
del song_total["Lyrics"]
del song_total["Date"]
del song_total["Genre_y"]
del song_total["Lyric_ver2"]
del song_total['Emotion_y']
del song_total['Proba']
del song_total["Lyric"]

In [12]:
song_total=song_total.rename(columns={"Genre_x":"Genre"})

In [13]:
song_total=song_total.rename(columns={"Emotion_x":"Emotion"})

In [14]:
song_total

,Title,SongID,Artist,Genre,Emotion,Probability,분노혐오,놀람공포,슬픔,행복,preprocess_Lyric_ver2
0,사랑하기 싫어,36206208,지아,발라드,슬픔,0.91,0.01,0.07,0.91,0.01,숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 ...
1,나비무덤,36235518,포맨 (4MEN),발라드,놀람공포,0.84,0.02,0.84,0.12,0.02,Standing beside you as sleep Wipe my tears as ...
2,사랑인가 봐,34657844,멜로망스,"발라드, 국내드라마",놀람공포,0.85,0.02,0.85,0.12,0.01,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...
3,너의 모든 순간,4446485,성시경,"발라드, 국내드라마",슬픔,0.66,0.02,0.30,0.66,0.02,이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 ...
4,사랑은 늘 도망가,34061322,임영웅,"발라드, 국내드라마",놀람공포,0.48,0.01,0.48,0.45,0.06,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...
...,...,...,...,...,...,...,...,...,...,...,...
3541,불티,36172278,최수호,성인가요/트로트,슬픔,0.95,0.02,0.02,0.95,0.01,나의 뜨거운 마음을 불같은 나의 마음을 다시 태울 수 없을까 헤어지기는 정말 싫어 ...
3542,안동역에서,32599795,임영웅,성인가요/트로트,놀람공포,0.92,0.02,0.92,0.03,0.03,바람에 날려버린 허무한 맹세였나 첫눈이 내리는 날 안동역 앞에서 만나자고 약속한사람...
3543,남행열차,83059,김수희,성인가요/트로트,놀람공포,0.72,0.01,0.72,0.20,0.07,비 내리는 호남선 남행열차에 흔들리는 차창 너머로 빗물이 흐르고 내 눈물도 흐르...
3544,킬리만자로의 표범,81123,조용필,성인가요/트로트,놀람공포,0.71,0.19,0.71,0.10,0.01,먹이를 찾아 산 기슭을 어슬렁 거리는 하이에나를 본일이 있는가 짐승의 썩은 고기만...


In [15]:
song=song_total.copy()

In [16]:
song["Genre"].value_counts()

댄스                      487
성인가요/트로트                471
랩/힙합                    432
R&B/Soul                415
발라드                     376
록/메탈                    328
인디음악, 포크/블루스            283
포크/블루스                  138
발라드, 인디음악               135
인디음악, 록/메탈              115
발라드, 국내드라마               73
R&B/Soul, 인디음악           68
랩/힙합, 인디음악               56
포크/블루스, 국내드라마            52
록/메탈, 국내드라마              40
R&B/Soul, 국내드라마          14
댄스, 랩/힙합                  7
인디음악, 일렉트로니카              6
발라드, R&B/Soul, 국내드라마      5
성인가요/트로트, 국내드라마           5
발라드, R&B/Soul             5
발라드, 국내영화                 4
발라드, 랩/힙합, 국내드라마          4
댄스, 인디음악                  4
록/메탈, 국내영화                3
록/메탈, 애니메이션/웹툰            3
발라드, 록/메탈                 3
랩/힙합, 국내드라마               2
발라드, 랩/힙합                 2
발라드, 포크/블루스               2
발라드, 성인가요/트로트             2
댄스, 애니메이션/웹툰              1
랩/힙합, 애니메이션/웹툰            1
포크/블루스, 국내영화              1
발라드, 인디음악, 포크/블루스         1
댄스, 국내드라마           

In [17]:
song["Emotion"].value_counts()

놀람공포    1493
행복       947
슬픔       729
분노혐오     377
Name: Emotion, dtype: int64

In [18]:
song[song["Emotion"]=="행복"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포   슬픔  \
Emotion Genre                                                               
행복      성인가요/트로트        168     168     168          168   168   168  168   
        댄스              118     118     118          118   118   118  118   
        발라드             103     103     103          103   103   103  103   
        랩/힙합             92      92      92           92    92    92   92   
        록/메탈             90      90      90           90    90    90   90   
        인디음악, 포크/블루스     80      80      80           80    80    80   80   

                       행복  preprocess_Lyric_ver2  
Emotion Genre                                     
행복      성인가요/트로트      168                    168  
        댄스            118                    118  
        발라드           103                    103  
        랩/힙합           92                     92  
        록/메탈           90                     90  
        인디음악, 포크/블루스   80                     80

In [19]:
song[song["Emotion"]=="놀람공포"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포   슬픔   행복  \
Emotion Genre                                                                
놀람공포    댄스          277     277     277          277   277   277  277  277   
        랩/힙합        246     246     246          246   246   246  246  246   
        R&B/Soul    217     217     217          217   217   217  217  217   
        록/메탈        140     140     140          140   140   140  140  140   
        성인가요/트로트    127     127     127          127   127   127  127  127   
        발라드         106     106     106          106   106   106  106  106   

                  preprocess_Lyric_ver2  
Emotion Genre                            
놀람공포    댄스                          277  
        랩/힙합                        246  
        R&B/Soul                    217  
        록/메탈                        140  
        성인가요/트로트                    127  
        발라드                         106

In [20]:
song[song["Emotion"]=="슬픔"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포   슬픔  \
Emotion Genre                                                               
슬픔      발라드             126     126     126          126   126   126  126   
        성인가요/트로트        111     111     111          111   111   111  111   
        R&B/Soul         86      86      86           86    86    86   86   
        인디음악, 포크/블루스     75      75      75           75    75    75   75   
        록/메탈             65      65      65           65    65    65   65   
        랩/힙합             46      46      46           46    46    46   46   

                       행복  preprocess_Lyric_ver2  
Emotion Genre                                     
슬픔      발라드           126                    126  
        성인가요/트로트      111                    111  
        R&B/Soul       86                     86  
        인디음악, 포크/블루스   75                     75  
        록/메탈           65                     65  
        랩/힙합           46                     46

In [21]:
song[song["Emotion"]=="분노혐오"].groupby(["Emotion","Genre"]).count().sort_values(by=["Title"],ascending=False).head(6)

Title  SongID  Artist  Probability  분노혐오  놀람공포  슬픔  행복  \
Emotion Genre                                                              
분노혐오    성인가요/트로트     65      65      65           65    65    65  65  65   
        랩/힙합         48      48      48           48    48    48  48  48   
        댄스           47      47      47           47    47    47  47  47   
        발라드          41      41      41           41    41    41  41  41   
        R&B/Soul     33      33      33           33    33    33  33  33   
        록/메탈         33      33      33           33    33    33  33  33   

                  preprocess_Lyric_ver2  
Emotion Genre                            
분노혐오    성인가요/트로트                     65  
        랩/힙합                         48  
        댄스                           47  
        발라드                          41  
        R&B/Soul                     33  
        록/메탈                         33

In [22]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_category = count_vect_category.fit_transform(song["Genre"]) 
song_simi_genre = cosine_similarity(genre_category, genre_category) 

In [23]:
cosine_similarity(genre_category, genre_category).mean()

0.13347373062194057

In [24]:
genre_result=pd.DataFrame(song_simi_genre)
genre_result.columns = song["Genre"]
genre_result.index = song["Genre"]
genre_result.head()

Genre,발라드,발라드,"발라드, 국내드라마","발라드, 국내드라마","발라드, 국내드라마",발라드,"발라드, 인디음악",발라드,발라드,"발라드, 인디음악",...,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트,성인가요/트로트
Genre,,,,,,,,,,,,,,,,,,,,,
발라드,1.00000,1.00000,0.57735,0.57735,0.57735,1.00000,0.577350,1.00000,1.00000,0.577350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
발라드,1.00000,1.00000,0.57735,0.57735,0.57735,1.00000,0.577350,1.00000,1.00000,0.577350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"발라드, 국내드라마",0.57735,0.57735,1.00000,1.00000,1.00000,0.57735,0.333333,0.57735,0.57735,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"발라드, 국내드라마",0.57735,0.57735,1.00000,1.00000,1.00000,0.57735,0.333333,0.57735,0.57735,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"발라드, 국내드라마",0.57735,0.57735,1.00000,1.00000,1.00000,0.57735,0.333333,0.57735,0.57735,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
vectorizer = TfidfVectorizer()
tf_lyric = vectorizer.fit_transform(song["preprocess_Lyric_ver2"]).todense().tolist()
song_simi_lyric = cosine_similarity(tf_lyric, tf_lyric) 

lyric_result=pd.DataFrame(song_simi_lyric)
lyric_result.columns = song["preprocess_Lyric_ver2"]
lyric_result.index = song["preprocess_Lyric_ver2"]

In [26]:
lyric_result.head()

preprocess_Lyric_ver2,숨을 내쉴 때마다 눈을 감을 때마다 니 생각이 나 먹먹해진다 하루 열두 시간이라면 스물네 시간의 반이면 외로움마저 반으로 줄 텐데 거지 같은 미련이 남아 추억을 주워 담아 사랑하고 싶어 하다가 주고 싶어 주다가 텅 빈 이내 맘 이별하기 싫어 울다가 받기 싫어 버텨보다가 결국 이렇게 나 혼자 거지 같은 미련이 남아 추억을 주워 담아 사랑하고 싶어 하다가 주고 싶어 주다가 텅 빈 이내 맘 이별하기 싫어 울다가 받기 싫어 버텨보다가 결국 이렇게 나 혼자 기나긴 시간이 지난 지금까지 못 잊어 그리워해 눈물 내일이면 그칠까 사랑하고 싶어 하다가 주고 싶어 주다가 텅 빈 이내 맘 이별하기 싫어 울다가 받기 싫어 버텨보다가 셀 수 없게 난 보기 싫게 난 결국 이렇게 나 혼자,Standing beside you as sleep Wipe my tears as I close the door 불러도 대답이 없어요 제발 날 불러봐요 아무 말도 안 할 거란 걸 알아 그렇게 못할 것도 알아 하지만 너 무슨 말을 할까 봐 조용히 귀를 열고서 아직 너의 집을 서성거리죠 아마도 예전처럼 날 보며 웃을 네 생각에 자꾸 잊게 되어가는 나 나는 너를 못 봐도 난 너와 아직 있는데 매일 네 생각 날 때면 이렇게 난 네 앞에 앉아 있어 세월 흘러가면서 난 자꾸 지워지겠지 이젠 더 이상 버티기 힘들어 너를 잊는 나를 용서해 내 눈물도 말라 가겠죠 이젠 더 이상 흐르지도 않죠 하지만 더 깊어가는 숨소리 내 곁에 멈춰 잠들어 하루 종일 너를 찾게 되는 걸 아마도 예전처럼 너 따뜻하진 않겠지만 그건 나의 운명이니까 나는 너를 못 봐도 나 항상 여기 있는데 매일 네 생각 날 때면 이렇게 난 네 앞에 앉아 있어 세월 흘러가면서 난 자꾸 지워가겠지 이젠 더 이상 버티기 힘들어 너를 잊는 나를 용서해 너를 이렇게 안아봐도 난 아무 느낌 없어 이젠 너 없는 건지 왜 그걸 모르지 넌 이미 여기엔 없는데 하늘이 갈라놓은 너와의 이별을 난 돌릴 순 없어 나 잊혀진 추억에 널 남겨둘게 나는 너를 못 봐도 나 항상 여기 있는데 매일 네 생각 날 때면 이렇게 난 네 앞에 앉아 있어 세월 흘러가면서 난 자꾸 지워가겠지 이젠 더 이상 버티기 힘들어 너를 잊는 나를 용서해 Standing beside you as sleep Wipe my tears as I close the door,너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습을 보고 있으면 자연스레 따라 웃고 있는 걸 너의 행동에 설레어하고 뒤척이다가 지새운 밤이 많아지는데 이건 누가 봐도 사랑일 텐데 종일 함께면 질릴 텐데 나 돌아서도 온통 너인 건 아무래도 사랑인가 봐 점점 너와 하고 싶은 일들 생각하면서 하룰 보낸 날이 많아지는데 이건 누가 봐도 사랑일 텐데 종일 함께면 질릴 텐데 나 돌아서도 온통 너인 건 아무래도 사랑인가 봐 너의 행복해하는 모습을 보고 있으면 나도 모르게 따라 웃는데 이 정도면 알아줄 만하잖아 너도 용기 낼만 하잖아 나만 이런 게 아니라면 우리 만나볼 만하잖아 아무래도 사랑인가 봐,이윽고 내가 한눈에 너를 알아봤을 때 모든 건 분명 달라지고 있었어 내 세상은 널 알기 전과 후로 나뉘어 니가 숨 쉬면 따스한 바람이 불어와 니가 웃으면 눈부신 햇살이 비춰 거기 있어줘서 그게 너라서 가끔 내 어깨에 가만히 기대주어서 나는 있잖아 정말 빈틈없이 행복해 너를 따라서 시간은 흐르고 멈춰 물끄러미 너를 들여다 보곤 해 그것 말고는 아무것도 할 수 없어서 너의 모든 순간 그게 나였으면 좋겠다 생각만 해도 가슴이 차올라 나는 온통 너로 보고 있으면 왠지 꿈처럼 아득한 것 몇 광년 동안 날 향해 날아온 별빛 또 지금의 너 거기 있어줘서 그게 너라서 가끔 나에게 조용하게 안겨주어서 나는 있잖아 정말 남김없이 고마워 너를 따라서 시간은 흐르고 멈춰 물끄러미 너를 들여다보곤 해 너를 보는 게 나에게는 사랑이니까 너의 모든 순간 그게 나였으면 좋겠다 생각만 해도 가슴이 차올라 나는 온통 너로 니 모든 순간 나였으면,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만 떨었지 내 아름답던 사람아 사랑이란 게 참 쓰린 거더라 잡으려 할수록 더 멀어지더라 이별이란 게 참 쉬운 거더라 내 잊지 못할 사람아 사랑아 왜 도망가 수줍은 아이처럼 행여 놓아버릴까 봐 꼭 움켜쥐지만 그리움이 쫓아 사랑은 늘 도망가 잠시 쉬어가면 좋을 텐데 바람이 분다 옷깃을 세워도 차가운 이별의 눈물이 차올라 잊지 못해서 가슴에 사무친 내 소중했던 사람아 사랑아 왜 도망가 수줍은 아이처럼 행여 놓아버릴까 봐 꼭 움켜쥐지만 그리움이 쫓아 사랑은 늘 도망가 잠시 쉬어가면 좋을 텐데 기다림도 애태움도 다 버려야 하는데 무얼 찾아 이 길을 서성일까 무얼 찾아 여기 있나 사랑아 왜 도망가 수줍은 아이처럼 행여 놓아버릴까 봐 꼭 움켜쥐지만 그리움이 쫓아 사랑은 늘 도망가 잠시 쉬어가면 좋을 텐데 잠시 쉬어가면 좋을 텐데,다 잊었다는 거짓말 또 해 버렸죠 내 마음에 그대란 사람 없다고 했죠 너무나 쉽게 잊혀졌다고 이젠 남이라고 서툰 내 사랑에 지쳐 떠난 그대 너무 많이 울려서 잡을 용기조차 낼 수 없었죠 미안해요 내 사랑아 다신 나 같은 사람 만나지 마요 혹시 찾아가도 두 번 다시 나를 허락해 주지 마요 그댄 여리고 너무 착해서 싫단 말도 잘 못하는데 많이 부족한 나를 사랑한 그대 이거면 됐어요 더 이상은 그대 불행하지 않도록 나 이쯤에서 없어지는 게 그댈 위한 나의 사랑인 걸요 너무 투명해서 때론 불안했죠 제멋대로 살아온 나를 감당할 수 없을 것 같죠 미안해요 내 사랑아 다신 나 같은 사람 만나지 마요 혹시 찾아가도 두 번 다시 나를 허락해 주지 마요 그댄 여리고 너무 착해서 싫단 말도 잘 못하는데 고마웠어요 나를 사랑해 줘서 나 같은 사람이 두 번 다시 감히 받을 수 없는 사랑 그대 때문에 행복했어요 울지 마요 그대 잘 생각해 봐요 나쁜 일들만 가득했던 우리 다 잊어야 해요 다신 나 같은 사람 만나지 마요 혹시 찾아가도 두 번 다시 나를 허락해 주지 마요 그댄 여리고 너무 착해서 싫단 말도 잘 못하는데 많이 부족한 나를 사랑한 그대 이거면 됐어요 더 이상은 그대 불행하지 않도록 나 이쯤에서 없어지는 게 그댈 위한 나의 사랑인 걸요,그녀와 나는요 그땐 참 어렸어요 많이 사랑했고 때론 많이 다퉜었죠 지금 생각하면 모두 내 잘못이죠 마지막 그날도 그녀는 울었어 나는 그녈 잡지 못했죠 지금까지도 너무 후회가 돼요 그 후로 누굴 사랑한 적 없어 아직은 그녀와 나는요 언젠가 만날 거죠 변해버린 모습 변해버린 시간 속에 하지만 괜찮아 내 눈엔 아름답던 예전 모습 그대로 일 거예요 그녀도 날 못 잊을 거야 나는 믿어요 그만큼 사랑했죠 그래서 우린 한 번은 만나야만 해요 그녀의 친구라도 이 노랠 듣는다면 그녀에게 전해줘요 내가 아직 사랑한다고 우리가 처음 만난 그 시간 그 자리에 내가 매일 기다린다고 언제라도 그녀도 날 못 잊을 거야 나는 믿어요 그만큼 사랑했죠 그래서 우린 한 번은 만나야만 해요 그녀의 친구라도 이 노랠 듣는다면 그녀에게 전해줘요 내가 아직 사랑한다고 우리가 처음 만난 그 시간 그 자리에 내가 매일 기다린다고 그녀의 친구라도 이 노랠 듣는다면 그녀에게 전해줘요 내가 아직 사랑한다고 우리가 처음 만난 그 시간 그 자리에 내가 매일 기다린다고 내가 매일 기다린다고,입을 맞춰도 불안하고 품에 안아도 초조하고 잠이 들 때도 꿈처럼 사라질까 밤새 뒤척이고 보면 볼수록 겁이 나고 겁이 날수록 더 보고 싶고 사랑할수록 네가 날 떠날까 봐 두려워지나 봐 사랑을 다 줘도 불안한 건 남자야 넌 너무 모르지 남자의 사랑을 사랑할수록 더 보고 싶고 보면 볼수록 더 안달 나고 평생 내 여자로 만들고픈 조급한 마음인걸 평생 내 여자로 살아줄래 나 말곤 없다고 말해줄래 조금도 불안해하지 않게 한 번 더 말해줄래 Say I love you Say I love you Say I love you Say I love you 사랑할수록 닮아가고 날이 갈수록 더 좋아져 함께 할수록 난 너 없인 못 살아 너 책임져 사랑을 다 줘도 불안한 건 남자야 넌 너무 모르지 남자의 사랑을 사랑할수록 더 보고 싶고 보면 볼수

In [28]:
song_simi_lyric.mean()

0.01254369097286877

In [29]:
song_simi_co = (
                 + song_simi_genre * 0.3 # 가중치 1. 장르 유사도
                 + song_simi_lyric * 1 # 가중치 2. 가사 유사도
                 )

In [30]:
song_simi_co_sorted_ind = song_simi_co.argsort()[:, ::-1] 
song_simi_co_sorted_df=pd.DataFrame(song_simi_co_sorted_ind)

In [31]:
# 행을 기준으로 가장 유사도 값이 큰 순으로 인덱스 저장
song_simi_co_sorted_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3536,3537,3538,3539,3540,3541,3542,3543,3544,3545
0,0,77,409,124,236,44,93,107,362,393,...,731,2294,3079,1567,725,2299,723,2301,2303,1397
1,1,227,204,195,411,90,224,370,79,277,...,3176,3178,3242,3182,3183,3241,3188,3441,3369,3204
2,2,4,298,3,72,128,112,157,191,307,...,2626,824,820,816,812,811,798,768,764,3545
3,3,3411,20,385,467,191,336,102,72,56,...,1818,609,3272,3271,3270,3269,608,607,606,930
4,4,342,2,192,430,323,112,20,378,191,...,1502,1503,1504,1506,1508,1510,1511,1512,1513,3545


In [33]:
song_simi_co_sorted_df.to_csv('song_similarity.csv')